In [1]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-37_5kp8d
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-37_5kp8d
     |████████████████████████████████| 132 kB 31.3 MB/s 
     |████████████████████████████████| 344 kB 30.0 MB/s 
     |████████████████████████████████| 49.1 MB 1.3 MB/s 
     |████████████████████████████████| 4.5 MB 60.5 MB/s 
     |████████████████████████████████| 1.2 MB 53.4 MB/s 
     |████████████████████████████████| 4.2 MB 45.3 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 49.6 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 79 kB 9.9 MB/s 
     |█████████

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split 

In [4]:
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [5]:
csv = pd.read_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/02_001_bert.csv') 
trainData, testData = train_test_split(csv, test_size = 0.3, random_state=42) 

In [6]:
csv

,Q,C,N
0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠,제품_불량_문의,39
1,마카롱 주문했는데 마카롱이 엄청 딱딱해요,제품_불량_문의,39
2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요,제품_불량_문의,39
3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요,제품_불량_문의,39
4,시루떡에서 왜 시큼한 냄새가 나는 거죠,제품_불량_문의,39
...,...,...,...
80106,기프티콘으로 결제 가능한 거 맞죠,결제_수단_문의,2
80107,카카오페이로 결제 된거 맞죠,결제_수단_문의,2
80108,모바일상품권도 결제할 수 있어요,결제_수단_문의,2
80109,기프티콘으로 음료 계산할 수 있는 거 맞죠,결제_수단_문의,2


In [7]:
category_list = list(csv['C'].unique())
category_list
cat_len = len(category_list)
print(cat_len)

63


In [8]:
df1 = pd.DataFrame(trainData)
df1.to_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/BERT/train/02_001_bert_train.tsv',index=False,
               sep='\t',encoding='utf-8')
df2 = pd.DataFrame(testData)
df2.to_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/BERT/test/02_001_bert_test.tsv',index=False,
               sep='\t',encoding='utf-8')



In [9]:
dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/BERT/train/02_001_bert_train.tsv", field_indices=[0,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/BERT/test/02_001_bert_test.tsv", field_indices=[0,2], num_discard_samples=1)

In [10]:
dataset_train[0]
dataset_test[0]


['네 포장해드릴까요 ', '57']

In [11]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [13]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5




In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [15]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
# print(data_train[0])
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [16]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=cat_len,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [18]:
import os

In [19]:
device = torch.device("cuda:0")
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [20]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [21]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [22]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [23]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [24]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/877 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 4.181766986846924 train acc 0.0
epoch 1 batch id 201 loss 3.5134835243225098 train acc 0.08146766169154229
epoch 1 batch id 401 loss 2.5243544578552246 train acc 0.17499220698254364
epoch 1 batch id 601 loss 1.757251262664795 train acc 0.3025946339434276
epoch 1 batch id 801 loss 0.820946216583252 train acc 0.39302434456928836
epoch 1 train acc 0.41981514779405316


  0%|          | 0/376 [00:00<?, ?it/s]

epoch 1 test acc 0.7263107204317897


  0%|          | 0/877 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 1.1218239068984985 train acc 0.78125
epoch 2 batch id 201 loss 1.3622004985809326 train acc 0.7131529850746269


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/03_DL_Model/02_001_bert_1.pth')
# torch.save(model, '/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/03_DeepLearning/bert(7,859)_1.pth')


In [ ]:
# !wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
# dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/BERT/train/bert(130,000)_train_1.tsv", field_indices=[0,2], num_discard_samples=1)

In [ ]:
# for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):

In [ ]:
new_test = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/BERT/train/02_001_bert_train.tsv", field_indices=[0,2], num_discard_samples=1)
# new_test = nlp.data.TSVDataset(".cache/ratings_train.txt" , field_indices=[0,2], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/02_001_bert.csv')
df

In [ ]:
category_list = list(df['C'].unique())
category_list
cat_len = len(category_list)
cat_len

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/03_DL_Model/02_001_bert_1.pth'))

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    if len(predict_sentence) <5 :
        print("너무 짧아요")
        return False
    else:




      data = [predict_sentence, '0']
      dataset_another = [data]

      another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
      test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=4)
      
      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)
          prediction = out.cpu().detach().numpy().argmax()
          print(prediction)

          test_eval=[]
          test_eval_2=[]
          for i in out:
              logits=i
              logits = logits.detach().cpu().numpy()
          for k , c in enumerate(category_list):
          # for k,(c,p) in enumerate(zip(category_list,prediction)):
              
              if np.argmax(logits) == k:
                  value = logits[k]
                  value  = round(value*10)
                  if value <50 :
                      print("죄송합니다 다시 말씀해주세요")
                      print(value)
                      # print()
                      test_eval.append(c)
                      value = logits[k]
                      test_eval_2.append(value)
                      print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")



                      return False
                  else :
                      test_eval.append(c)
                      value = logits[k]
                      test_eval_2.append(value)

              # elif np.argmax(logits) == 1:
              #     test_eval.append("텀블라사용")

          print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
          print(test_eval)
          print(test_eval_2[0])


end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요(5자 이상) : ")
    if sentence == 0 :
        
        break
    predict(sentence)


    print("\n")




In [ ]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요(5자 이상) : ")
    if sentence == 0 :
        
        break
    predict(sentence)


    print("\n")